In [ ]:
import pickle
import json
from glob import glob

In [ ]:
#pkl_list = glob("./gqa_data/*.pkl")

with open("./gqa_data/created_data.pkl" , "rb") as f:
    pkl = pickle.load(f)
    
with open("./gqa_data/train_balanced_questions.json") as f:
    questions = json.load(f)

In [ ]:
from pprint import pprint

#pprint(pkl['2354786'])

pprint(pkl['2354786'][5])

#print(questions['2354786'])

In [ ]:
import json
from glob import glob
from pprint import pprint
from tqdm import tqdm

In [ ]:
with open("./gqa_data/train_balanced_questions.json") as f:
    questions = json.load(f)

In [ ]:
with open("./sceneGraphs/train_sceneGraphs.json") as f:
    sceneGraphs = json.load(f)

In [ ]:
def get_related_object_names(objects, related_objects, question):
    
    object_ids = {}
    
    question_annotated_objects = related_objects['question']
    '''
    for key, value in related_objects.items():
    
        object_ids[key] = set()
        for start_idx, object_id  in value.items():
            object_ids[key].add(object_id)
    '''
    object_ids['question'] = set()
    for start_idx, object_id in question_annotated_objects.items():
       
        object_ids['question'].add(object_id)
    
    object_names = []
    
    related_object_dict = {}
    related_object_ids = set()
    related_object_names = set()
    irrelated_object_dict = {}
    
    for key, object_ids in object_ids.items():
        related_object_dict[key] = {}
        for object_id in object_ids:
            object = objects[object_id]
            object_name = object['name']
            if object_name in question:
                related_object_dict[key][object_id] = object_name
                related_object_ids.add(object_id)
                related_object_names.add(object_name)
    
    for key, value in objects.items():
        
        object_name = value['name']
        if not key in related_object_ids and object_name in related_object_names:
            irrelated_object_dict[key] = object_name
    
    
    return related_object_dict, irrelated_object_dict

In [ ]:
print(type(questions))

ambiguous_questions = {}
exsit_question_nums = 0

for idx, (question_key, question_value) in tqdm(enumerate(questions.items())):
    if idx < 1:
        pprint(question_value)
    #pprint(question_value)
    if 'exist' in question_value['types']['detailed']:
        exsit_question_nums += 1
        # if exsit_question_nums < 5:
        #    print("exist-question\n",question_value['question'])
        continue
    image_id = question_value['imageId']
    #pprint(sceneGraphs[image_id])
    sceneGraph=sceneGraphs[image_id]
    related_objects = question_value['annotations']
    objects = sceneGraph['objects']
    
    related_object_names, irrelated_object_names = get_related_object_names(objects, related_objects,question_value['question'] )
    #names_set = set(names)
    
    #if len(names) != len(names_set):
    if len(irrelated_object_names) > 0:
        if len(ambiguous_questions.keys()) < 30:
        #pprint(question_value)
        #pprint(sceneGraphs[image_id])
            print(image_id)
            pprint(related_object_names)
            pprint(irrelated_object_names)
            print(question_value['question'])
            print(question_value['fullAnswer'])
            print(question_value['types']['detailed'])
            print("---------------------------")
        #print('다름')
        #break
        question_value['irrelated_object_names'] = irrelated_object_names
        ambiguous_questions[question_key] = question_value

In [ ]:
print(len(questions.keys()))
print(exsit_question_nums)
print(len(ambiguous_questions.keys()))

In [ ]:
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl",cache_dir="./blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", torch_dtype=torch.float16, cache_dir="./blip2-flan-t5-xxl")
model.to(device)

In [ ]:
new_dataset = {}
new_dataset_length = len(ambiguous_questions)
for idx, (question_id, question_value) in tqdm(enumerate(ambiguous_questions.items()), total=new_dataset_length):
    if idx == new_dataset_length:
        break
    
    image_id = question_value['imageId']
    img_file = './images/' + image_id + '.jpg' 
    raw_image = Image.open(img_file).convert('RGB')

    question = "Instructions: Given a picture, A question and a correct answer related the picture are provided.\
    The answer can be inferred from the picture. \
    The target(object) of question  which is existed in the picture is important key to infer the answer. (objects in question are bold) \
    The additional information of object related question is helpful for answer the question more correctly. \
    Therefore, our goal is to get new information related answer by asking a new question. \
    Write an additional question to help to answer the original question correctly.  \
    original question: " + question_value['question'] + " " + \
    "uncertain information: " + list(question_value['irrelated_object_names'].values())[0] + " " + \
    "answer: " + question_value['fullAnswer'] + " " + \
    "additional question: "

    if idx < 2:
        print(question)

    inputs = processor(raw_image, question, return_tensors="pt").to(device, torch.float16)
    out = model.generate(**inputs)
    additional_question = processor.decode(out[0], skip_special_tokens=True)
    
    question_value['addtional_question'] = additional_question
    new_dataset[question_id] = question_value
    
    

In [ ]:
with open("ambiguous_questions.json", 'w') as f:
    json.dump(new_dataset, f, ensure_ascii=False)

In [ ]:
import json
import random
import numpy as np

with open("ambiguous_questions.json", 'r') as f:
    full_dataset  = json.load(f)

dataset_sampling = {}

full_list = list(full_dataset.items())
np.random.seed(42)
random_indexs = np.random.choice(range(0, len(full_list)), 200, replace = False)
print(random_indexs)

In [ ]:
import csv

train_list = [['question_id','question', 'answer', 'intermediate_question', 'ambiquous_object', 'image_id']]
test_list = [['question_id','question', 'answer', 'intermediate_question', 'ambiquous_object', 'image_id']]
for idx, q_idx in enumerate(random_indexs):
    (key, value) = full_list[q_idx] 
    dataset_sampling[key] = value
    original_q = value['question']
    answer = value['fullAnswer']
    ambiguous_object = list(value['irrelated_object_names'].values())[0]
    intermediate_q = value["addtional_question"]
    image_id = value['imageId']
    
    if idx < 100:
        train_list.append([q_idx, original_q, answer, intermediate_q, ambiguous_object, image_id])
    else:
        test_list.append([q_idx, original_q, answer, intermediate_q, ambiguous_object, image_id])
    
with open("ambiguous_questions_100_random.csv", 'w') as f:
#     json.dump(dataset_sampling, f, ensure_ascii=False)
    writer = csv.writer(f)
    writer.writerows(train_list)
    
with open("ambiguous_questions_test.csv", 'w') as f:
#     json.dump(dataset_sampling, f, ensure_ascii=False)
    writer = csv.writer(f)
    writer.writerows(test_list)

In [ ]:
import csv
from datasets import load_dataset

dataset = load_dataset('csv',data_files={'train': "./ambiguous_questions_train.csv", 'test' : "./ambiguous_questions_test.csv"})

print(dataset['train'])

#with open("ambiguous_questions_100_random_label.csv", 'r') as f:
#    reader = csv.reader(f)
    
#    for line in reader:
#        print(line)
    

In [ ]:
import pandas as pd
import json
import numpy as np

few_shot_examples = {}

intermediate_questions =  pd.read_excel("intermediate_questions_samples_32.xlsx", dtype={'qid' : str})

with open("./ambiguous_questions_rebuilt.json") as f:
    ambiguous_questions = json.load(f)

with open("./ambiguous_questions.json") as f:
    original_questions = json.load(f)

print(len(ambiguous_questions))
for idx in intermediate_questions.index:
    intermediate_question_example = intermediate_questions.iloc[idx]
    qid = intermediate_question_example['qid']

    ambiguous_question_example = ambiguous_questions[qid]
    
    ambiguous_question = ambiguous_question_example['question']
    intermediate_question = intermediate_question_example['intermediate question']
    original_question_example = original_questions.pop(qid)
    
    few_shot_examples[qid] = original_question_example
    few_shot_examples[qid]['ambiguous_question'] = ambiguous_question
    few_shot_examples[qid]['intermediate_question'] = intermediate_question
    few_shot_examples[qid].pop("addtional_question")
    
    entities_list = set()
    for object_name in few_shot_examples[qid]['irrelated_object_names'].values():
        if object_name in few_shot_examples[qid]['ambiguous_question']:
             entities_list.add(object_name)
    few_shot_examples[qid]['question_entities'] = list(entities_list)
   
np.random.seed(42) 
test_keys = np.random.choice(list(original_questions.keys()), 32, replace=False)
test_examples = {}

for qid in test_keys:
    original_questions[qid].pop("addtional_question")
    test_examples[qid] =  original_questions[qid]
    
    ambiguous_question_example = ambiguous_questions[qid]
    ambiguous_question = ambiguous_question_example['question']
    
    test_examples[qid]["ambiguous_question"] = ambiguous_question
    
    entities_list = set()
    for object_name in test_examples[qid]['irrelated_object_names'].values():
        if object_name in test_examples[qid]['ambiguous_question']:
             entities_list.add(object_name)
    test_examples[qid]['question_entities'] = list(entities_list)

In [ ]:
from pprint import pprint
test_examples_list = [{k:v}  for k,v in test_examples.items()]
pprint(test_examples_list[0])